In [11]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
import pandas as pd
from connect_oracle import criar_conexao

In [13]:
df = pd.read_excel("superstore.xlsx", sheet_name="Orders")

In [14]:
df.Category.unique()

array(['Furniture', 'Office Supplies', 'Technology'], dtype=object)

In [42]:
connection = criar_conexao()
cursor = connection.cursor()

In [16]:
cursor.execute("""
    begin
        execute immediate 'drop table products';
        exception when others then if sqlcode <> -942 then raise; end if;
    end;""")

In [18]:
cursor.execute("""
    create table products (
               Product_ID varchar2(15),
               Category varchar2(20),
               Sub_Category varchar2(20),
               Product_Name varchar2(100),
               primary key (Product_ID))""")

In [19]:
df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [30]:
products = df[["Product ID", "Category", "Sub-Category", "Product Name"]]

In [31]:
products.columns

Index(['Product ID', 'Category', 'Sub-Category', 'Product Name'], dtype='object')

In [32]:
products.rename(
    columns={
        'Product ID': 'Product_ID',
        'Sub-Category': 'Sub_Category',
        'Product Name': 'Product_Name'}, inplace=True)

In [33]:
products.drop_duplicates(subset = 'Product_ID', inplace=True, ignore_index=True)

In [34]:
products.columns

Index(['Product_ID', 'Category', 'Sub_Category', 'Product_Name'], dtype='object')

In [37]:

query = []
for index, row in products.iterrows():
    query += [(
        str(products.Product_ID[index]),
        str(products.Category[index]),
        str(products.Sub_Category[index]),
        str(products.Product_Name[index])
    )]

In [38]:
print(query[1:3])

[('FUR-CH-10000454', 'Furniture', 'Chairs', 'Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back'), ('OFF-LA-10000240', 'Office Supplies', 'Labels', 'Self-Adhesive Address Labels for Typewriters by Universal')]


In [43]:
cursor.executemany("""
        insert into products (
            Product_ID, 
            Category, 
            Sub_Category, 
            Product_Name) values(:1, :2, :3, :4)"""
            , query)
print(cursor.rowcount, "Rows Inserted")

1862 Rows Inserted


In [44]:
connection.commit()

In [45]:
connection.close()